In [1]:
#!/usr/bin/env python
# -*- encoding: utf-8 -*-
'''
@File    :   Original_Split .ipynb
@Time    :   2025/05/08 10:49:52
@Author  :   Neutrin 
'''

# here put the import lib

import pandas as pd 


In [3]:
# 定义掩码CSV文件和乳腺X光片CSV文件的路径
csv_mask_path = 'E:/Dataset/CBIS-DDSM/New_CBDDSM/roi/csv'
csv_mam_path = 'E:/Dataset/CBIS-DDSM/New_CBDDSM/img/csv'

# 创建包含两种数据集路径的字典
files = {'Mam_Set':csv_mam_path,'Mask_Set':csv_mask_path}
# 定义要处理的两种图像类型（钙化和肿块）
type_dict = ['Calc-Training','Mass-Training']


final_dfs = dict()  # 存储最终处理后的数据框
for path in files.keys():  # 遍历两种数据集（乳腺X光片和掩码）
    dfs=[]  # 临时存储当前数据集类型的所有数据框
    for i in type_dict:  # 遍历两种图像类型（钙化和肿块）

        # 读取CSV文件并将"pathology"列重命名为"label"
        data = pd.read_csv(files[path]+'/'+i.lower()+'.csv')
        data = data.rename(columns={"pathology": "label"})

        # 对钙化图像类型进行标签编码：恶性=1, 良性=2
        if i.startswith("Calc"):
            labels = {"MALIGNANT":1 , "BENIGN":2}
            data["label"]= data.label.map(labels)
        # 对肿块图像类型进行标签编码：恶性=3, 良性=4
        else:
            labels = {"MALIGNANT":3 , "BENIGN":4}  
            data["label"]= data.label.map(labels)

        dfs.append(data)  # 将处理后的数据添加到列表

    # 合并同一数据集类型下的所有数据框
    final_df = pd.concat(dfs)
    
    # 将合并后的数据框存入字典
    final_dfs[path] = final_df

In [4]:
# 处理乳腺X光片数据集
mam_df = final_dfs['Mam_Set'].reset_index(drop= True)  # 重置索引
final_mam_df = mam_df.drop(columns=["img"])  # 删除图像ID列
# 保存乳腺X光片训练集CSV
final_mam_df.to_csv(csv_mam_path+'/'+'official_mam_train_set.csv',index = False)

# 重命名列，准备与掩码数据合并
mam_df = mam_df.rename(columns={"img": "merge","label":"label_mam"})

# 处理掩码数据集
mask_df = final_dfs['Mask_Set'].reset_index(drop= True)  # 重置索引

# 创建匹配列：从掩码图像ID中去掉后两个字符（通常是"_1"或"_2"）
# 这样可以将掩码与对应的原始乳腺X光片匹配
mask_df['merge'] = mask_df.apply(lambda row: row['img'][:-2],
                                         axis=1) 

# 基于匹配列合并乳腺X光片和掩码数据
new_mask_df = pd.merge(mask_df, mam_df, how='inner',
                                     on='merge')

# 清理合并后的数据框，删除不需要的列
new_mask_df = new_mask_df.drop(columns=["img","merge","label_mam"])

# 重新排列列的顺序
new_mask_df = new_mask_df.iloc[:,[2,0,1]]

# 保存掩码训练集CSV
new_mask_df.to_csv("official_mask_train_set.csv",index=False)